In [1]:
import requests
import json
import sys

sys.path.append('..')

# Data Manipulation
import pandas as pd

from utils import *

# Agregados

In [2]:
url = 'https://servicodados.ibge.gov.br/api/v3/agregados'
df_agregated = make_df(get_request_json(url))
df_agregated

,id,nome,agregados
0,D5,Áreas Urbanizadas,"[{'id': '8418', 'nome': 'Áreas urbanizadas, Lo..."
1,CL,Cadastro Central de Empresas,"[{'id': '1732', 'nome': 'Dados gerais das empr..."
2,CA,Censo Agropecuário,"[{'id': '1278', 'nome': 'Agroindústria rural n..."
3,ME,"Censo Comum do Mercosul, Bolívia e Chile","[{'id': '2059', 'nome': 'Domicílios e Populaçã..."
4,CD,Censo Demográfico,"[{'id': '1301', 'nome': 'Área e Densidade demo..."
...,...,...,...
62,VS,Produção da Extração Vegetal e da Silvicultura,"[{'id': '5930', 'nome': 'Área total existente ..."
63,PO,Produção de Ovos de Galinha,"[{'id': '915', 'nome': 'Número de informantes,..."
64,IO,Produto Interno Bruto dos Municípios,"[{'id': '599', 'nome': 'Índice de Gini do prod..."
65,XE,Projeção da População,"[{'id': '7362', 'nome': 'Esperança de vida ao ..."


In [9]:
research_ids = df_agregated['id']
print(list(research_ids))

['D5', 'CL', 'CA', 'ME', 'CD', 'CM', 'DU', 'DT', 'C4', 'DR', 'DZ', 'SU', 'ST', 'DE', 'XF', 'FP', 'IU', 'IL', 'IR', 'IJ', 'PC', 'IA', 'IQ', 'IG', 'PG', 'LA', 'C2', 'AC', 'PB', 'PY', 'PP', 'AM', 'ES', 'PM', 'IT', 'OF', 'SH', 'SE', 'PT', 'RC', 'PI', 'PK', 'PJ', 'DG', 'PZ', 'XY', 'AB', 'MC', 'PE', 'SC', 'PL', 'SB', 'XN', 'AA', 'PD', 'B5', 'BB', 'DD', 'AX', 'CQ', 'LT', 'PA', 'VS', 'PO', 'IO', 'XE', 'SI']


In [44]:
%%time
df_aux = pd.DataFrame()

for research_id in research_ids:
    # get just the row for the given research id
    df_research = df_agregated[df_agregated['id'] == research_id]
    # explode the row to get all the agregados 
    df_research = df_research.explode('agregados')
    # reset index
    df_research = df_research.reset_index()
    # normalize the agregados column and rename the columns
    df_normalize = pd.json_normalize(df_research['agregados'])
    df_normalize = df_normalize.rename({'id': 'id-agregado', 'nome': 'nome-agregado'})
    # concat the agregado data to df_research
    df_research = pd.concat([df_research, df_normalize], axis = 1)
    # delete the dict column
    df_research = df_research.drop('agregados', axis = 1)


CPU times: total: 219 ms
Wall time: 218 ms


In [45]:
df_research

,index,id,nome,id,nome
0,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,33,"Custo de projeto m², por padrão de acabamento ..."
1,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,647,"Custo de projeto m², por tipo de projeto e pad..."
2,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,30,Custo médio m² em moeda corrente (série encerr...
3,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,2296,Custo médio m² em moeda corrente e variações p...
4,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,6586,Custo médio m² em moeda corrente e variações p...
5,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,32,"Custo médio m² em variação percentual, por tip..."
6,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,34,"Preços medianos, por materiais e serviços (sér..."
7,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,2062,"Preços medianos, por materiais e serviços (sér..."
8,66,SI,Sistema Nacional de Pesquisa de Custos e Índic...,35,"Salários medianos, por categorias profissionai..."
